## Day 21: Monkey Math

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day21/puzzle.ipynb)

### Preparation

It promises to be convenient to build a calculation tree. The input is read to the dictionary first, then the tree is recursively built.

In [81]:
#!value --name sampleRaw
root: pppw + sjmn
dbpl: 5
cczh: sllz + lgvd
zczc: 2
ptdq: humn - dvpt
dvpt: 3
lfqf: 4
humn: 5
ljgn: 2
sjmn: drzm * dbpl
sllz: 4
pppw: cczh / lfqf
lgvd: ljgn * ptdq
drzm: hmdt - zczc
hmdt: 32

In [82]:
type Op = string
type MonkeyBillet = 
    | Num of int64
    | Op of string * Op * string

In [83]:
#load "../common/common.fsx"

let parse s = 
    Pattern1.read id s 
    |> Array.map (
        function 
        | Regex "(\w+): (\d+)" [monkey; num] ->
            monkey, int64 num |> MonkeyBillet.Num
        | Regex "(\w+): (\w+) (.) (\w+)" [monkey; x; op; y] ->
            monkey, MonkeyBillet.Op (x, op, y)
    ) |> Map.ofArray


In [84]:
#!share sampleRaw --from value

parse sampleRaw

key,type,Item1,Item2,Item3,Item
cczh,FSI_0138+MonkeyBillet+Op,sllz,+,lgvd,
dbpl,FSI_0138+MonkeyBillet+Num,,,,5
drzm,FSI_0138+MonkeyBillet+Op,hmdt,-,zczc,
dvpt,FSI_0138+MonkeyBillet+Num,,,,3
hmdt,FSI_0138+MonkeyBillet+Num,,,,32
humn,FSI_0138+MonkeyBillet+Num,,,,5
lfqf,FSI_0138+MonkeyBillet+Num,,,,4
lgvd,FSI_0138+MonkeyBillet+Op,ljgn,*,ptdq,
ljgn,FSI_0138+MonkeyBillet+Num,,,,2
pppw,FSI_0138+MonkeyBillet+Op,cczh,/,lfqf,


In [85]:
type YellMonkey = { Id: string; Number: int64 }
and WaitMonkey = { Id: string; Left: MonkeyNode; Op: Op; Right: MonkeyNode }
and MonkeyNode = 
    | Yell of YellMonkey
    | Wait of WaitMonkey

let monkeyId monkey = match monkey with | Yell { Id = id } | Wait { Id = id } -> id
    
let rec buildTree id (dic : Map<string, MonkeyBillet>) = 
    match dic[id] with
    | MonkeyBillet.Num number 
        -> Yell { Id = id; Number = number }
    | MonkeyBillet.Op (left, op, right) 
        -> Wait { Id = id; Left = buildTree left dic; Op = op; Right = buildTree right dic }

In [86]:
let sampleMonkeys = sampleRaw |> parse |> buildTree "root"
sampleMonkeys

Item
"WaitMonkey Id: root Left: Wait Item: WaitMonkey Id: pppw Left: Wait Item: WaitMonkey Id: cczh Left: Yell Item: { Id = ""sllz"" Number = 4L } Op: + Right: Wait Item: { Id = ""lgvd"" Left = Yell { Id = ""ljgn"" Number = 2L } Op = ""*"" Right = Wait { Id = ""ptdq"" Left = Yell { Id = ""humn"" Number = 5L } Op = ""-"" Right = Yell { Id = ""dvpt"" Number = 3L } } } Op: / Right: Yell Item: YellMonkey Id: lfqf Number: 4 Op: + Right: Wait Item: WaitMonkey Id: sjmn Left: Wait Item: WaitMonkey Id: drzm Left: Yell Item: { Id = ""hmdt"" Number = 32L } Op: - Right: Yell Item: { Id = ""zczc"" Number = 2L } Op: * Right: Yell Item: YellMonkey Id: dbpl Number: 5"


### Part 1

With the tree built, part 1 is trivial. The tree is traversed and node values are calculated in a depth-first manner.

In [87]:
let rec calculate node = 
    match node with
    | Yell { Number = num } -> num
    | Wait { Left = left; Op = op; Right = right } ->
        let leftResult = calculate left
        let rightResult = calculate right
        let f = 
            match op with
            | "*" -> (*)
            | "+" -> (+)
            | "/" -> (/)
            | "-" -> (-)
        f leftResult rightResult

In [88]:
calculate sampleMonkeys

152

For the actual input:

In [89]:
#!value --name actualRaw --from-file ./data_actual.txt

In [111]:
#!share actualRaw --from value

let actualMonkeys = actualRaw |> parse |> buildTree "root"
actualMonkeys |> calculate

324122188240430

### Part 2

For path 2 we want to find the path to the `humn` node first. 

In [91]:
type PathItem = | Left | Right

In [92]:
let rec findPath root target = 
    if monkeyId root = target then Some []
    else 
        match root with
        | Yell _ -> None
        | Wait { Left = left; Right = right } -> 
            match findPath left target with
            | Some found -> Some (Left :: found)
            | None -> 
                match findPath right target with
                | Some found -> Some (Right :: found)
                | None -> None

In [100]:
let sampleHumnPath = findPath sampleMonkeys "humn" |> Option.get
sampleHumnPath |> List.map string

index,value
0,Left
1,Left
2,Right
3,Right
4,Left


Now, going down the tree along this path, on each step we can calculate the other subtree value and determine the target value for the subtree with the `humn` node.

In [105]:
let rec private calculateHumn' path tree target = 
    match path with 
    | (current::rest) -> 
        match tree with
        | Wait {Left = left; Op = op; Right = right} -> 
            match current, op with
            | Left, "+" -> // x + right = target => x = target - right
                calculateHumn' rest left (target - (calculate right))
            | Left, "-" -> // x - right = target => x = target + right
                calculateHumn' rest left (target + (calculate right))
            | Left, "*" -> // x * right = target => x = target / right
                calculateHumn' rest left (target / (calculate right))
            | Left, "/" -> // x / right = target => x = target * right
                calculateHumn' rest left (target * (calculate right))
            | Right, "+" -> // left + x = target => x = target - left
                calculateHumn' rest right (target - (calculate left))
            | Right, "-" -> // left - x = target => x = left - target
                calculateHumn' rest right ((calculate left) - target)
            | Right, "*" -> // left * x = target => x = target / left
                calculateHumn' rest right (target / (calculate left))
            | Right, "/" -> // left / x = target => x = left / target
                calculateHumn' rest right ((calculate left) / target)
    | _ -> target

let calculateHumn path tree = 
    match path with 
    | (current::rest) -> 
        match tree with
        | Wait {Left = left; Right = right} -> 
            match current with
            | Left -> // x = right
                calculateHumn' rest left (calculate right)
            | Right -> // left = x
                calculateHumn' rest right (calculate left)



In [112]:
calculateHumn sampleHumnPath sampleMonkeys

301

In [109]:
calculateHumn (findPath actualMonkeys "humn" |> Option.get) actualMonkeys

3412650897405